<a href="https://colab.research.google.com/github/Mouna876/Mouna876-Classification-des-Images/blob/main/PFA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Chemins des répertoires
train_dir = "/content/drive/MyDrive/chest_xray/chest_xray/train"
val_dir = "/content/drive/MyDrive/chest_xray/chest_xray/val"
test_dir = "/content/drive/MyDrive/chest_xray/chest_xray/test"

In [ ]:
# Paramètres
img_height, img_width = 224, 224
batch_size = 8

In [ ]:
# Créer des générateurs de données avec augmentation pour l'entraînement et sans augmentation pour la validation et le test
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = val_test_datagen.flow_from_directory(
    directory=val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)

Found 5240 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 634 images belonging to 2 classes.


In [ ]:
# Charger MobileNetV2 pré-entraîné sans les couches supérieures
base_model = MobileNetV2(input_shape=(img_height, img_width, 3), weights='imagenet', include_top=False)

9406464/9406464 [==============================] - 1s 0us/step


In [ ]:
# Congeler les couches du modèle de base
base_model.trainable = False

In [ ]:
# Ajouter des couches de classification au modèle
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [ ]:
# Créer le modèle final
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Compiler le modèle
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Entraîner le modèle
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
655/655 [==============================] - 2689s 4s/step - loss: 0.1923 - accuracy: 0.9181 - val_loss: 0.2993 - val_accuracy: 0.9375
Epoch 2/10
655/655 [==============================] - 331s 504ms/step - loss: 0.1541 - accuracy: 0.9366 - val_loss: 0.4159 - val_accuracy: 0.8125
Epoch 3/10
655/655 [==============================] - 316s 483ms/step - loss: 0.1451 - accuracy: 0.9406 - val_loss: 0.3209 - val_accuracy: 0.8750
Epoch 4/10
655/655 [==============================] - 330s 503ms/step - loss: 0.1365 - accuracy: 0.9447 - val_loss: 0.2527 - val_accuracy: 0.8125
Epoch 5/10
655/655 [==============================] - 328s 501ms/step - loss: 0.1165 - accuracy: 0.9523 - val_loss: 0.2591 - val_accuracy: 0.8750
Epoch 6/10
655/655 [==============================] - 317s 485ms/step - loss: 0.1265 - accuracy: 0.9510 - val_loss: 0.3042 - val_accuracy: 0.8750
Epoch 7/10
655/655 [==============================] - 326s 497ms/step - loss: 0.1108 - accuracy: 0.9550 - val_loss: 0.4183 - v

In [ ]:
# Évaluer le modèle sur l'ensemble de test
evaluation = model.evaluate(test_generator)
print("Accuracy on test set:", evaluation[1] * 100, "%")

80/80 [==============================] - 310s 4s/step - loss: 0.2665 - accuracy: 0.8959
Accuracy on test set: 89.58990573883057 %


In [ ]:
model.save('/content/drive/MyDrive/ColabNotebooks/chest_xray_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
